## Задание 9.5. Модуль ML-6 (HW-03)

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.

**Этапы выполнения:**

* Выделить три столбца-признака для обучения, выбранные RFE.
* Выделить три столбца-признака для обучения, выбранные SelectKBest.
* Обучить регрессию на первых трёх столбцах, оценив качество модели на тесте.
* Обучить регрессию на вторых трёх столбцах, оценив качество модели на тесте.
* Произвести сравнение выбранных метрик в форме комментария. Дать ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?»

In [1]:
# загружаем библиотеки
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
#импортируем данные
data = pd.read_excel('data/data_ford_price.xlsx')

In [3]:
# выполняеем предобработку данных, удаляем пропуски, отделяем результирующий признак от остальных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)
y = data['price']
x = data.drop(columns='price')

In [4]:
# делим датасет на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### **Метод рекурсивного исключения признаков (RFE)**

In [5]:
from sklearn.feature_selection import RFE

In [7]:
# выделим три наиболее значимых признака:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [8]:
# оставим только нужные признаки
x = data[['year', 'cylinders', 'lat']]

# делим датасет на тренировочную и тестовую выборки с учётом нового признака
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

# оцениваем метрики
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5206.677


### **Выбор k лучших переменных: SelectKBest**

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression

In [12]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)
y = data['price']
x = data.drop(columns='price')

# делим датасет на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# выбираем оптимальные признаки
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [13]:
# оставляем только нужные признаки
x = data[['year', 'cylinders', 'odometer']]

# делим датасет на тренировочную и тестовую выборки с учётом нового признака
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

# оцениваем метрику
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4800.916


**Вывод:**

**В ходе исследования было выявлено, что метод SelectKBest показал лучший результат на тестовой выборке. Он позволил достичь более низкого значения метрики MAE по сравнению с RFE. Отличие в результатах методов заключалось только в одном признаке: SelectKBest отметил 'odometer' на третьем месте, в то время как RFE выбрал 'lat'. Дальнейший анализ позволил сделать вывод о том, что пробег ('odometer') оказывает значительно большее влияние на цену автомобиля, чем координаты расположения ('lat'). Это подтверждается средней абсолютной ошибкой модели регрессии, обученной на первых трех столбцах, выбранных методом RFE, которая составила 5206.677. В то же время, модель, обученная на вторых трех столбцах, выбранных методом SelectKBest, показала меньшую среднюю абсолютную ошибку - 4800.916. Таким образом, на основании полученных результатов можно сделать вывод, что метод отбора признаков SelectKBest демонстрирует лучшие результаты на тестовой выборке. Меньшая средняя абсолютная ошибка указывает на более точное предсказание цены автомобиля. Это свидетельствует о том, что выбранные признаки, определенные методом SelectKBest, являются более информативными и релевантными для построения модели регрессии.**